In [8]:
import requests
import json

DEMOGRAPHY_URL = "https://api.precisely.com/demographics-segmentation/v1/demographics/bylocation"

def get_demography_data(latitude, longitude, api_key):
    params = {
        'latitude': latitude,
        'longitude': longitude
    }

    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    
    response = requests.get(DEMOGRAPHY_URL, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error", response.text)
        response.raise_for_status()

api_key = 'Y58vnO2TOA1rwN0InhrJLYAADvQsKm4p' 
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))

try:
    demography_data = get_demography_data(latitude, longitude, api_key)
    print(json.dumps(demography_data, indent=4))
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Error {"errors":[{"errorCode":"PB-APIM-ERR-1002","errorDescription":"Invalid Access Token"}]}
Error: 401 Client Error: Unauthorized for url: https://api.precisely.com/demographics-segmentation/v1/demographics/bylocation?latitude=35.0118&longitude=-81.9571


In [15]:
import requests
from xml.etree import ElementTree

def get_topography_data(longitude, latitude):
    bbox = f"{longitude},{latitude},{longitude},{latitude}"

    base_url = "https://portal.opentopography.org/geoportal/csw/discovery"
    params = {
        "Request": "GetRecords",
        "Service": "CSW",
        "Version": "2.0.2",
        "resultType": "results",
        "TypeNames": "csw:Record",
        "ElementSetName": "full",
        "BBOX": bbox
    }

    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        parse_xml(response.content)
    else:
        print(f"Error: {response.status_code}")

def parse_xml(xml_content):
    root = ElementTree.fromstring(xml_content)
    namespace = {"ns0": "http://www.opengis.net/cat/csw/2.0.2", "dc": "http://purl.org/dc/elements/1.1/", "ns3": "http://www.opengis.net/ows"}

    for record in root.findall(".//ns0:Record", namespaces=namespace):
        title = record.find(".//dc:title", namespaces=namespace)
        file_id = record.find(".//dc:identifier[@scheme='urn:x-esri:specification:ServiceType:ArcIMS:Metadata:FileID']", namespaces=namespace)
        doc_id = record.find(".//dc:identifier[@scheme='urn:x-esri:specification:ServiceType:ArcIMS:Metadata:DocID']", namespaces=namespace)
        lower_corner = record.find(".//ns3:LowerCorner", namespaces=namespace)
        upper_corner = record.find(".//ns3:UpperCorner", namespaces=namespace)

       
        print(f"Title: {title.text if title is not None else 'N/A'}")
        print(f"Identifier (FileID): {file_id.text if file_id is not None else 'N/A'}")
        print(f"Identifier (DocID): {doc_id.text if doc_id is not None else 'N/A'}")
        print(f"Bounding Box (WGS84) - Lower Corner: {lower_corner.text if lower_corner is not None else 'N/A'}")
        print(f"Bounding Box (WGS84) - Upper Corner: {upper_corner.text if upper_corner is not None else 'N/A'}")
        print("------------------------")

if __name__ == "__main__":
    longitude = - 85.3131 
    latitude = 25.0961     
    get_topography_data(longitude, latitude)


Title: Drone lidar survey of the San Juan Fault, Vancouver Island, September 2022
Identifier (FileID): OTDS.102023.3157.2
Identifier (DocID): {65A9D0B3-2836-4CFA-A685-CBF4A285E5E0}
Bounding Box (WGS84) - Lower Corner: -123.881458654248 48.6114724880793
Bounding Box (WGS84) - Upper Corner: -123.812815277922 48.6292087871735
------------------------
Title: Survey of a subtle scarp at Chon Kemin, Kyrgyzstan, July 2019
Identifier (FileID): OTDS.102023.32643.4
Identifier (DocID): {9E0330A6-FF2A-4A5A-85D8-689C92417F62}
Bounding Box (WGS84) - Lower Corner: 76.5904742144722 42.8623492881219
Bounding Box (WGS84) - Upper Corner: 76.6010195577865 42.8676929988822
------------------------
Title: Survey of a prominent thrust scarp at Chon Kemin, Kyrgyzstan, July 2019
Identifier (FileID): OTDS.102023.32643.3
Identifier (DocID): {960A5E02-CB8F-4FF6-9429-3388DC0A74CB}
Bounding Box (WGS84) - Lower Corner: 76.5653372628732 42.8544413312498
Bounding Box (WGS84) - Upper Corner: 76.5791798270853 42.8614594

In [10]:
import requests

def get_elevation(lat, lon, api_key):
    """
    Get elevation for specific latitude and longitude using Google Maps Elevation API.
    
    Parameters:
    - lat (float): Latitude
    - lon (float): Longitude
    - api_key (str): Your Google Maps Elevation API key
    
    Returns:
    - dict: A dictionary with elevation data or error message
    """

    endpoint = "https://maps.googleapis.com/maps/api/elevation/json"
    params = {
        "locations": f"{lat},{lon}",
        "key": api_key
    }
    
    response = requests.get(endpoint, params=params).json()
    
    if response.get('status') == "OK":
        return response['results'][0]
    else:
        return {"error": response.get('status')}

if __name__ == "__main__":
    # Set your Google Maps Elevation API key here
    API_KEY = "YOUR_GOOGLE_MAPS_API_KEY"
    
    # Test the function
    latitude = float(input("Enter the latitude: "))
    longitude = float(input("Enter the longitude: "))
    
    elevation_data = get_elevation(latitude, longitude, API_KEY)
    
    if "error" in elevation_data:
        print("Error:", elevation_data["error"])
    else:
        print(f"Elevation at ({latitude}, {longitude}): {elevation_data['elevation']} meters")



In [20]:
import requests
from bs4 import BeautifulSoup

def fetch_google_results_for_location(lat, lon):
    query = f"topography, demography, and market trends at {lat}, {lon}"
    url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    search_results = []
    
    for g in soup.find_all('div', class_='tF2Cxc'):
        title = g.find('h3')
        if title:
            title = title.text

        link = g.find('a')
        if link:
            link = link['href']

        description = g.find('span', class_='aCOpRe')
        if description:
            description = description.text

        if title and link and description:
            search_results.append((title, description, link))
    
    return search_results

if __name__ == "__main__":
    try:
        lat = float(input("Enter the latitude: "))
        lon = float(input("Enter the longitude: "))
        results = fetch_google_results_for_location(lat, lon)
        
        print("\nSearch Results:\n")
        for index, (title, description, link) in enumerate(results, 1):
            print(f"Result {index}:")
            print(f"Place Name (Assuming it's part of the title): {title}")
            print(f"Description: {description}")
            print(f"Link: {link}")
            print("-" * 50)
    except Exception as e:
        print(f"An error occurred: {e}")



Search Results:



In [22]:
pip install spacy


     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     ---------------------------------------- 45.9/45.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/158.6 kB ? eta -:--:--
     -------------------------- ----------- 112.6/158.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 158.6/158.6 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.6 kB ? eta -:--:--
     -------------------------------------- 181.6/181.6 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.3/12.1 MB 7.9 MB/s eta 0:00:02
   - -------------------------------------- 0.6/12.1 MB 5.9 MB/s eta 0:00:02
   - -------------------------------------- 0.6/12.1 MB 6.0 MB/s eta 0:00:02
   - ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import requests
from bs4 import BeautifulSoup
import spacy

# Load the spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def fetch_google_results_for_location(lat, lon):
    query = f"topography, demography, and market trends at {lat}, {lon}"
    url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    search_results = []
    
    for g in soup.find_all('div', class_='tF2Cxc'):
        title = g.find('h3')
        if title:
            title = title.text

        link = g.find('a')
        if link:
            link = link['href']

        description = g.find('span', class_='aCOpRe')
        if description:
            description = description.text
            doc = nlp(description)  # Moved inside the condition
            places = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
        else:
            places = []

        if title and link and description:
            search_results.append({
                "title": title,
                "description": description,
                "link": link,
                "places": places
            })
    
    return search_results

if __name__ == "__main__":
    try:
        lat = float(input("Enter the latitude: "))
        lon = float(input("Enter the longitude: "))
        results = fetch_google_results_for_location(lat, lon)
        
        print("\nSearch Results:\n")
        for index, result in enumerate(results, 1):
            print(f"Result {index}:")
            print(f"Title: {result['title']}")
            print(f"Description: {result['description']}")
            print(f"Link: {result['link']}")
            print(f"Identified Places: {', '.join(result['places'])}")
            print("-" * 50)
    except Exception as e:
        print(f"An error occurred: {e}")



Search Results:

